<a href="https://colab.research.google.com/github/ihshin227/tensorlite_convert_test/blob/master/keras_attention_convert_to_tensorflow_lite_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import os
import unicodedata
import re
import io
import time
from sklearn.model_selection import train_test_split

In [0]:
def build_model(input_layer, start_neurons):
  lstm = tf.keras.layers.LSTM(start_neurons, return_sequences=True)(input_layer)
  attention = tf.keras.layers.Attention()([lstm, lstm])
  output_layer = tf.keras.layers.Dense(units=1)(attention)

  return output_layer

In [0]:
input_layer = tf.keras.layers.Input((18, 18), batch_size=32)
output_layer = build_model(input_layer=input_layer, start_neurons=4)

In [0]:
model = tf.keras.models.Model(input_layer, output_layer)

In [0]:
model.compile(optimizer='sgd', loss='mean_squared_error')

In [0]:
inputs = np.random.random([32, 18, 18]).astype(np.float32)
targets = np.random.rand(32).astype(np.float32)

model.fit(inputs, targets, epochs=20)

Epoch 1/20
1/1 [==============================] - 0s 2ms/step - loss: 1.6579
Epoch 2/20
1/1 [==============================] - 0s 1ms/step - loss: 1.4589
Epoch 3/20
1/1 [==============================] - 0s 920us/step - loss: 1.2769
Epoch 4/20
1/1 [==============================] - 0s 964us/step - loss: 1.1086
Epoch 5/20
1/1 [==============================] - 0s 924us/step - loss: 0.9513
Epoch 6/20
1/1 [==============================] - 0s 1ms/step - loss: 0.8030
Epoch 7/20
1/1 [==============================] - 0s 1ms/step - loss: 0.6633
Epoch 8/20
1/1 [==============================] - 0s 902us/step - loss: 0.5348
Epoch 9/20
1/1 [==============================] - 0s 2ms/step - loss: 0.4224
Epoch 10/20
1/1 [==============================] - 0s 870us/step - loss: 0.3313
Epoch 11/20
1/1 [==============================] - 0s 1ms/step - loss: 0.2626
Epoch 12/20
1/1 [==============================] - 0s 1ms/step - loss: 0.2134
Epoch 13/20
1/1 [==============================] - 0s 1ms/step 

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [0]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)


In [0]:
interpreter.invoke()

RuntimeError: ignored

In [0]:
class simple_attention(tf.keras.Model):
  def __init__(self, start_neurons):
    super(simple_attention, self).__init__()
    self.lstm = tf.keras.layers.LSTM(start_neurons, return_sequences=True)
    self.attention = tf.keras.layers.Attention()
    self.dense = tf.keras.layers.Dense(units=1)

  def call(self, x):
    out1 = self.lstm(x)
    out2 = self.attention([out1, out1])
    out3 = self.dense(out2)

    return out3
at = simple_attention(4)

# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.LSTM(4, return_sequences=True))
# model.add(tf.keras.layers.LSTM(4))
# model.add(tf.keras.layers.Dense(units=1, input_shape=[32,4]))
# model.compile(optimizer='sgd', loss='mean_squared_error')

seq = tf.keras.models.Sequential()
seq.add(tf.keras.layers.LSTM(4, return_sequences=True))
seq.add(tf.keras.layers.Attention())
seq.add(tf.keras.layers.Dense(units=1))

In [0]:
at.compile(optimizer='sgd', loss='mean_squared_error')
at.fit(inputs, targets, epochs=20)

Epoch 1/20
1/1 [==============================] - 0s 2ms/step - loss: 0.1759
Epoch 2/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1669
Epoch 3/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1591
Epoch 4/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1523
Epoch 5/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1463
Epoch 6/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1411
Epoch 7/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1364
Epoch 8/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1322
Epoch 9/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1285
Epoch 10/20
1/1 [==============================] - 0s 864us/step - loss: 0.1252
Epoch 11/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1222
Epoch 12/20
1/1 [==============================] - 0s 973us/step - loss: 0.1194
Epoch 13/20
1/1 [==============================] - 0s 1ms/step - loss

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(at)
tflite_model = converter.convert()

In [0]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [0]:
interpreter.invoke()

RuntimeError: ignored

In [0]:
tflite_results = interpreter.get_tensor(output_details[0]['index'])
tf_results = model(tf.constant(input_data))

In [0]:
a = model(input_data)
print(a.shape)
print(a)

In [0]:
b = model(inputs)
print(b.shape)

In [0]:
print(input_details[0]['index'])

In [0]:
print(input_details[0]['shape'])

In [0]:
print(input_data.shape)